In [6]:
from pathlib import Path
from typing import Union, Literal, List
from PyPDF2 import PdfWriter, PdfReader


reader = PdfReader("../debu.pdf")
writer = PdfWriter()
for index in range(reader.numPages):
    content_page = reader.pages[index]
    mediabox = content_page.mediabox

    # You need to load it again, as the last time it was overwritten
    reader_stamp = PdfReader("../gdgKolkata.pdf")
    image_page = reader_stamp.pages[0]

    image_page.merge_page(content_page)
    image_page.mediabox = mediabox
    writer.add_page(image_page)

with open("output.pdf", 'wb') as output_file:
    # Write the output PDF file
    writer.write(output_file)


In [ ]:
from pathlib import Path
from typing import Union, Literal, List
from PyPDF2 import PdfWriter, PdfReader
import os
import mimetypes
import random
from io import BytesIO
import shutil


def main():
    global download_location, not_removed
    download_location, not_removed = pdf_stamper(
        'pdf-encryptor', convertables)
    not_removed = not_convertable.to_py()+not_removed
    print(type(download_location), type(not_removed))
    if download_location:
        mime = mimetypes.guess_type(download_location)[0]
        print(mime)
        with open(download_location, "rb") as f:
            file_content = f.read()
        downloadFile(file_content, download_location.split('/')[-1], mime)


def file_uid_generator(path_name, destination_type):
    charset = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    while True:
        fuid = ''
        for i in range(5):
            fuid += random.choice(charset)
        if not os.path.exists(f"{path_name}_{fuid}.{destination_type}"):
            return fuid


def empty_root_folder():
    for item in os.listdir():
        if os.path.isfile(item):
            os.remove(item)
        else:
            shutil.rmtree(item)


def pdf_resizer(folder_name, convertables):
    empty_root_folder()
    os.makedirs(f'{folder_name}')
    not_removed = []
    print(len(convertables))
    if len(convertables):
        length = len(convertables)
        for i in range(length):
            input_file, input_bin_str = convertables[i][0]
            stamp_file, stamp_bin_str = convertables[i][1]
            input_bytes_obj = BytesIO(
                bytes(input_bin_str, encoding="raw_unicode_escape"))
            stamp_bytes_obj = BytesIO(
                bytes(stamp_bin_str, encoding="raw_unicode_escape"))
            try:
                reader = PdfReader(input_bytes_obj)
                writer = PdfWriter()
                for index in range(reader.numPages):
                    content_page = reader.pages[index]
                    mediabox = content_page.mediabox
                    reader_stamp = PdfReader(stamp_bytes_obj)
                    image_page = reader_stamp.pages[0]

                    image_page.merge_page(content_page)
                    image_page.mediabox = mediabox
                    writer.add_page(image_page)
                filename = input_file[::-1].split('.', maxsplit=1)[-1][::-1]
                path_without_extension = f"{folder_name}/{filename}"
                fuid = '' if not os.path.exists(
                    path_without_extension+".pdf") else '_'+file_uid_generator(path_without_extension, "pdf")

                with open(f'{path_without_extension+fuid}.pdf', "wb") as f:
                    writer.write(f)

            except Exception as e:
                print("entered exception")
                not_removed.append(f"{input_file}")
                print(e)
                continue
    else:
        not_removed = [input_file for input_file,
                       input_bin_str in convertables]
        print("length mismatch")

    convertables_in_dir = [f for f in os.listdir(
        f"{folder_name}") if f"{f}"[0].isalnum()]
    print(convertables_in_dir)
    total_files = len(convertables_in_dir)
    downloadable_location = None
    if total_files > 1:
        shutil.make_archive(f"{folder_name}", "zip", f"{folder_name}")
        shutil.rmtree(f"{folder_name}")
        downloadable_location = f"{folder_name}.zip"
    elif total_files == 1:
        downloadable_location = f"{folder_name}/{convertables_in_dir[0]}"
    else:
        shutil.rmtree(f"{folder_name}")
    print("not_converted: ", not_removed)
    return [downloadable_location, not_removed]


main()
